In [1]:
import pandas as pd
import numpy as np
x =  pd.read_csv("micro_high_res_every_frame.csv")
y = pd.read_csv("/content/updated_fer.csv")


1) Compute model averaged sentiment scores for each individual sentiment type from different models (FER, DeepFace etc)


In [2]:
fer_mean = y.mean(axis=0)
deep_mean = x[["df_angry","df_disgust","df_fear","df_happy","df_sad","df_surprise","df_neutral"]].mean(axis=0)
fer_mean = pd.DataFrame(fer_mean)
deep_mean = pd.DataFrame(deep_mean)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [3]:
fer_mean=fer_mean.iloc[1:]
fer_mean=fer_mean.T
print(fer_mean)

      angry   disgust      fear     happy       sad  surprise   neutral
0  0.121618  0.003862  0.074554  0.141433  0.170354  0.036661  0.449863


In [4]:
deep_mean = deep_mean.T
print(deep_mean)

   df_angry  df_disgust    df_fear   df_happy     df_sad  df_surprise  \
0   7.47512    0.134416  15.583923  12.856685  18.135053     3.090492   

   df_neutral  
0   42.724311  


In [5]:
avg_modal = pd.DataFrame(np.vstack([fer_mean, deep_mean]), columns = list(fer_mean.columns))
avg_modal.head()


,angry,disgust,fear,happy,sad,surprise,neutral
0,0.121618,0.003862,0.074554,0.141433,0.170354,0.036661,0.449863
1,7.475120,0.134416,15.583923,12.856685,18.135053,3.090492,42.724311


In [6]:
avg_modal = avg_modal.mean(axis=0)
print(avg_modal)

angry        3.798369
disgust      0.069139
fear         7.829239
happy        6.499059
sad          9.152703
surprise     1.563576
neutral     21.587087
dtype: float64


2) Define positivity score for each frame (row) in each video at 24 fps from the average of smiling and happiness scores from different models. (rows)


In [7]:
fer = y[['angry',   'disgust',      'fear',     'happy',       'sad',  'surprise',   'neutral']]
deep =  x[["df_angry","df_disgust","df_fear","df_happy","df_sad","df_surprise","df_neutral"]]
fer.head()

,angry,disgust,fear,happy,sad,surprise,neutral
0,0.15,0.0,0.13,0.00,0.11,0.02,0.59
1,0.11,0.0,0.02,0.60,0.23,0.00,0.04
2,0.45,0.0,0.02,0.01,0.04,0.01,0.48
3,0.07,0.0,0.01,0.53,0.03,0.00,0.36
4,0.01,0.0,0.01,0.12,0.02,0.00,0.85


In [8]:
deep.head()

,df_angry,df_disgust,df_fear,df_happy,df_sad,df_surprise,df_neutral
0,6.900348e-08,5.559958e-17,99.991345,3.681116e-06,0.000002,2.090444e-05,0.008619
1,2.158997e-01,2.428260e-04,91.233181,5.492600e+00,0.010351,3.043159e+00,0.004567
2,1.597073e-08,1.978551e-18,99.993324,8.619106e-07,0.000010,5.335329e-06,0.006659
3,7.836559e-08,6.851789e-18,99.994034,1.133913e-06,0.000008,9.646076e-06,0.005951
4,4.976388e-10,3.948474e-22,23.895361,6.848739e-07,0.004213,5.436169e-08,76.100427


In [9]:
deep = deep.rename({'df_happy': 'happy','df_angry':'angry','df_disgust':'disgust','df_fear':'fear','df_sad':'sad','df_surprise':'surprise','df_neutral':'neutral'}, axis=1)
deep.head()

,angry,disgust,fear,happy,sad,surprise,neutral
0,6.900348e-08,5.559958e-17,99.991345,3.681116e-06,0.000002,2.090444e-05,0.008619
1,2.158997e-01,2.428260e-04,91.233181,5.492600e+00,0.010351,3.043159e+00,0.004567
2,1.597073e-08,1.978551e-18,99.993324,8.619106e-07,0.000010,5.335329e-06,0.006659
3,7.836559e-08,6.851789e-18,99.994034,1.133913e-06,0.000008,9.646076e-06,0.005951
4,4.976388e-10,3.948474e-22,23.895361,6.848739e-07,0.004213,5.436169e-08,76.100427


In [10]:
positive_score=pd.concat([deep[["happy"]],fer[["happy"]]],axis=1)
positive_score.head()

,happy,happy
0,3.681116e-06,0.00
1,5.492600e+00,0.60
2,8.619106e-07,0.01
3,1.133913e-06,0.53
4,6.848739e-07,0.12


In [11]:
positive_score['positive_score'] = positive_score.mean(axis=1)
positive_score.head()

,happy,happy,positive_score
0,3.681116e-06,0.00,0.000002
1,5.492600e+00,0.60,3.046300
2,8.619106e-07,0.01,0.005000
3,1.133913e-06,0.53,0.265001
4,6.848739e-07,0.12,0.060000


Compute negativity score from the average of sadness, anger, disgust, contempt and fear scores from different models for each frame (row) in each video at 24 fps. (row)

In [12]:
negative_score=pd.concat([deep[["angry","disgust","sad","fear"]],fer[["angry","disgust","sad","fear"]]],axis=1)
negative_score['negative_score'] = negative_score.mean(axis=1)
negative_score.head()

,angry,disgust,sad,fear,angry,disgust,sad,fear,negative_score
0,6.900348e-08,5.559958e-17,0.000002,99.991345,0.15,0.0,0.11,0.13,12.547668
1,2.158997e-01,2.428260e-04,0.010351,91.233181,0.11,0.0,0.23,0.02,11.477459
2,1.597073e-08,1.978551e-18,0.000010,99.993324,0.45,0.0,0.04,0.02,12.562917
3,7.836559e-08,6.851789e-18,0.000008,99.994034,0.07,0.0,0.03,0.01,12.513005
4,4.976388e-10,3.948474e-22,0.004213,23.895361,0.01,0.0,0.02,0.01,2.992447


Compute face.sentiment score for each frame (row) by dividing the positivity score by the sum of the positive + negative sentiment scores. This score shows the percentage of net face sentiment that is positive (0% to 100%).


In [1]:
face_sentiment_score = positive_score['positive_score'] / negative_score['negative_score'] + positive_score['positive_score']

NameError: ignored